In [6]:
#Part 1

class Switch:
    def __init__(self,width,pos):
        self.pressed = False
        self.width = width
        self.pos=pos
        self.radius = width//4
        self.font = pygame.font.Font(None,width//3)
        
        self.wholerect = pygame.Rect(pos,(width,width//2))
        self.middlerect = pygame.Rect((pos[0]+width//4,pos[1]),(width//2,width//2))
        self.middlecolor = 'Red'
        
        self.movingcirclepos = [self.pos[0]+3*self.radius,self.pos[1]+self.radius]
        self.movement = 0
        self.direction = -5
        
        self.leftcircle = pygame.draw.circle(screen,'Red',(pos[0]+self.radius,pos[1]+self.radius),self.radius,0)
        self.rightcircle = pygame.draw.circle(screen,'Green',(pos[0]+3*self.radius,pos[1]+self.radius),self.radius,0)
        print(self.leftcircle.center)
        print(self.rightcircle.center)
        #test out to see if you can remove
        self.textoff_surf = self.font.render('Off',True,'black')
        self.texton_surf = self.font.render('On',True,'black')
        #might be wrrongg because circle isn't a surface
        self.textoff_rect =  self.textoff_surf.get_rect(center = (pos[0]+self.radius,pos[1]+self.radius))
        self.texton_rect = self.texton_surf.get_rect(center = (pos[0]+3*self.radius,pos[1]+self.radius))
       
        
    def draw(self):
        if self.movingcirclepos[0] == self.pos[0]+3*self.radius:
             self.direction = -abs(self.direction)
        elif self.movingcirclepos[0] == self.pos[0]+self.radius:
            self.direction = abs(self.direction)
        if self.movement != 0:
            self.movingcirclepos[0] += self.direction
            self.movement -= abs(self.direction)  
        
        pygame.draw.rect(screen,self.middlecolor, self.middlerect)
        
        pygame.draw.circle(screen,'Red',(self.pos[0]+self.radius,self.pos[1]+self.radius),self.radius,0)
        pygame.draw.circle(screen,'Green',(self.pos[0]+3*self.radius,self.pos[1]+self.radius),self.radius,0)
        
        screen.blit(self.textoff_surf, self.textoff_rect)
        screen.blit(self.texton_surf, self.texton_rect) 
        
        pygame.draw.circle(screen,'#979797',self.movingcirclepos ,self.radius,0)
        self.check_click()
        
    def check_click(self):
        global switch1status
        mouse_pos = pygame.mouse.get_pos()
        if self.wholerect.collidepoint(mouse_pos):
            if pygame.mouse.get_pressed()[0]:
                self.pressed = True
            else:
                if self.pressed == True:
                    self.movement = self.width//2
                    if self.middlecolor == 'Red':
                        self.middlecolor = 'Green'
                    else: 
                        self.middlecolor = 'Red'
                    self.pressed = False
                    
        if self.middlecolor == 'Red':
            switch1status = 'Off'
        else:
            switch1status = 'On'
                    
with open('24.txt', 'r') as f:
    data=tuple(line for line in f.read().splitlines())

winds=[]
setofwinds=set()
permissiblelocations=set()
permissiblelocations.add((1,0))
permissiblelocations.add((len(data[-1])-2,len(data)-1))

#print(permissiblelocations)

for j in range(1,len(data)-1):
    for i in range(1,len(data[j])-1):
        if data[j][i] in ["<","^","v",">"]:
            winds.append([(i,j),data[j][i]])
        permissiblelocations.add((i,j))

#Will simiulatie having a one elf. This elf will undergo mitosis and turn into 5 elves. Each of these elveves will then move up,down,left,right or still. Some will die
#the ones that prevail will multiply into 5 again. Until we get one elf that ends up at the goal. 

survivingelves=set()
survivingelves.add((1,0))
def movewind():
    global setofwinds
    global winds
    setofwinds={}
    for i in range(len(winds)):
        relocatemap={"<":(winds[i][0][0]-1,winds[i][0][1]),
                    ">":(winds[i][0][0]+1,winds[i][0][1]),
                    "^":(winds[i][0][0],winds[i][0][1]-1),
                    "v":(winds[i][0][0],winds[i][0][1]+1)
                    }
        relocate=relocatemap[winds[i][1]]
        if relocate not in permissiblelocations:
            relocatemap={"<":(len(data[-1])-2,winds[i][0][1]),
                        ">":(1,winds[i][0][1]),
                        "^":(winds[i][0][0],len(data)-2),
                        "v":(winds[i][0][0],1)
                        }
            relocate=relocatemap[winds[i][1]]
        #print(relocate)
        winds[i][0]=relocate
        if relocate not in setofwinds:
            setofwinds[relocate] = winds[i][1]
        else:
            if type(setofwinds[relocate])==int:
                setofwinds[relocate]+=1
            else: setofwinds[relocate] = 2
done=False
time=0

import pygame
from sys import exit
pygame.init()

SCREEN_WIDTH = 1800
SCREEN_HEIGHT = 800

screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption('Advent of code 2022 day24 part1')
clock=pygame.time.Clock()
font = pygame.font.SysFont(None, 22)
font2 = pygame.font.SysFont(None, 18)
font3 = pygame.font.SysFont(None, 30)
surface = pygame.Surface((SCREEN_WIDTH,SCREEN_HEIGHT))
surface.fill("#400000")
switch1=Switch(80,(300,640))
switch1status = False



def draw():
    global setofwinds
    global survivingelves
    global data
    global surface
    global time
    global switch1status
    screen.blit(surface,(0,0))
    string=""
    jj=20
    ii=10
    for j in range(len(data)):
        for i in range(len(data[-1])):
            if (i,j) in survivingelves:
                string+=r"@"
                img = font2.render(string, True, "pink")
            elif (i,j) in permissiblelocations:
                if switch1status=="Off":
                    string+="-"
                else:
                    if (i,j) in setofwinds:
                        string+=str(setofwinds[(i,j)])
                    else:
                        string+="-"
                if string!="-": img = font2.render(string, True, "green")
                else: img = font.render(string, True, "green")
            else:
                string+="#"
                img = font2.render(string, True, "green")
        
            screen.blit(img, (ii, jj))
            string=""
            ii+=14
        jj+=20
        ii=10
    img = font3.render(f"Time: {time}", True, "white")
    img2 = font3.render(f"Turn on wind directions: ", True, "white")
    screen.blit(img, (50, 600))
    screen.blit(img2, (50, 650))


def wait(x):
    for i in range(x):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                exit()
        switch1.draw()  
        pygame.display.update()
        clock.tick(60)
        
while not done and time<1000:
    time+=1
    movewind()
    survivingelves2=set()
    for i in survivingelves:
        location=i
        locations=((location[0]+1,location[1]),(location[0]-1,location[1]),(location[0],location[1]+1),(location[0],location[1]-1),location)
        for j in locations:
            if j not in setofwinds and j in permissiblelocations:
                elf=(j)
                survivingelves2.add(elf)
                if j==(len(data[-1])-2,len(data)-1):
                    done=elf
            
    survivingelves=survivingelves2
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            exit()
    draw()
    switch1.draw()   
    pygame.display.update()
    wait(4)
    
print(done)     
print(time)   

while True:
    wait(10000)
    


(320, 660)
(360, 660)
(120, 26)
277


SystemExit: 

In [7]:
#Part 2

class Switch:
    def __init__(self,width,pos):
        self.pressed = False
        self.width = width
        self.pos=pos
        self.radius = width//4
        self.font = pygame.font.Font(None,width//3)
        
        self.wholerect = pygame.Rect(pos,(width,width//2))
        self.middlerect = pygame.Rect((pos[0]+width//4,pos[1]),(width//2,width//2))
        self.middlecolor = 'Red'
        
        self.movingcirclepos = [self.pos[0]+3*self.radius,self.pos[1]+self.radius]
        self.movement = 0
        self.direction = -5
        
        self.leftcircle = pygame.draw.circle(screen,'Red',(pos[0]+self.radius,pos[1]+self.radius),self.radius,0)
        self.rightcircle = pygame.draw.circle(screen,'Green',(pos[0]+3*self.radius,pos[1]+self.radius),self.radius,0)
        print(self.leftcircle.center)
        print(self.rightcircle.center)
        #test out to see if you can remove
        self.textoff_surf = self.font.render('Off',True,'black')
        self.texton_surf = self.font.render('On',True,'black')
        #might be wrrongg because circle isn't a surface
        self.textoff_rect =  self.textoff_surf.get_rect(center = (pos[0]+self.radius,pos[1]+self.radius))
        self.texton_rect = self.texton_surf.get_rect(center = (pos[0]+3*self.radius,pos[1]+self.radius))
       
        
    def draw(self):
        if self.movingcirclepos[0] == self.pos[0]+3*self.radius:
             self.direction = -abs(self.direction)
        elif self.movingcirclepos[0] == self.pos[0]+self.radius:
            self.direction = abs(self.direction)
        if self.movement != 0:
            self.movingcirclepos[0] += self.direction
            self.movement -= abs(self.direction)  
        
        pygame.draw.rect(screen,self.middlecolor, self.middlerect)
        
        pygame.draw.circle(screen,'Red',(self.pos[0]+self.radius,self.pos[1]+self.radius),self.radius,0)
        pygame.draw.circle(screen,'Green',(self.pos[0]+3*self.radius,self.pos[1]+self.radius),self.radius,0)
        
        screen.blit(self.textoff_surf, self.textoff_rect)
        screen.blit(self.texton_surf, self.texton_rect) 
        
        pygame.draw.circle(screen,'#979797',self.movingcirclepos ,self.radius,0)
        self.check_click()
        
    def check_click(self):
        global switch1status
        mouse_pos = pygame.mouse.get_pos()
        if self.wholerect.collidepoint(mouse_pos):
            if pygame.mouse.get_pressed()[0]:
                self.pressed = True
            else:
                if self.pressed == True:
                    self.movement = self.width//2
                    if self.middlecolor == 'Red':
                        self.middlecolor = 'Green'
                    else: 
                        self.middlecolor = 'Red'
                    self.pressed = False
                    
        if self.middlecolor == 'Red':
            switch1status = 'Off'
        else:
            switch1status = 'On'
                    
with open('24.txt', 'r') as f:
    data=tuple(line for line in f.read().splitlines())

winds=[]
setofwinds=set()
permissiblelocations=set()
permissiblelocations.add((1,0))
permissiblelocations.add((len(data[-1])-2,len(data)-1))

#print(permissiblelocations)

for j in range(1,len(data)-1):
    for i in range(1,len(data[j])-1):
        if data[j][i] in ["<","^","v",">"]:
            winds.append([(i,j),data[j][i]])
        permissiblelocations.add((i,j))

#Will simiulatie having a one elf. This elf will undergo mitosis and turn into 5 elves. Each of these elveves will then move up,down,left,right or still. Some will die
#the ones that prevail will multiply into 5 again. Until we get one elf that ends up at the goal. 

survivingelves={}
survivingelves[(1,0)]=0
def movewind():
    global setofwinds
    global winds
    setofwinds={}
    for i in range(len(winds)):
        relocatemap={"<":(winds[i][0][0]-1,winds[i][0][1]),
                    ">":(winds[i][0][0]+1,winds[i][0][1]),
                    "^":(winds[i][0][0],winds[i][0][1]-1),
                    "v":(winds[i][0][0],winds[i][0][1]+1)
                    }
        relocate=relocatemap[winds[i][1]]
        if relocate not in permissiblelocations:
            relocatemap={"<":(len(data[-1])-2,winds[i][0][1]),
                        ">":(1,winds[i][0][1]),
                        "^":(winds[i][0][0],len(data)-2),
                        "v":(winds[i][0][0],1)
                        }
            relocate=relocatemap[winds[i][1]]
        #print(relocate)
        winds[i][0]=relocate
        if relocate not in setofwinds:
            setofwinds[relocate] = winds[i][1]
        else:
            if type(setofwinds[relocate])==int:
                setofwinds[relocate]+=1
            else: setofwinds[relocate] = 2
done=False
time=0

import pygame
from sys import exit
pygame.init()

SCREEN_WIDTH = 1800
SCREEN_HEIGHT = 800

screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption('Advent of code 2022 day24 part 2')
clock=pygame.time.Clock()
font = pygame.font.SysFont(None, 22)
font2 = pygame.font.SysFont(None, 18)
font3 = pygame.font.SysFont(None, 30)
surface = pygame.Surface((SCREEN_WIDTH,SCREEN_HEIGHT))
surface.fill("#400000")
switch1=Switch(80,(300,640))
switch1status = False



def draw():
    global setofwinds
    global survivingelves
    global data
    global surface
    global time
    global switch1status
    screen.blit(surface,(0,0))
    string=""
    jj=20
    ii=10
    for j in range(len(data)):
        for i in range(len(data[-1])):
            if (i,j) in survivingelves:
                string+=r"@"
                if survivingelves[(i,j)]==0:
                    img = font2.render(string, True, "pink")
                elif survivingelves[(i,j)]==1:
                    img = font2.render(string, True, "blue")
                else:
                    img = font2.render(string, True, "purple")
            elif (i,j) in permissiblelocations:
                if switch1status=="Off":
                    string+="-"
                else:
                    if (i,j) in setofwinds:
                        string+=str(setofwinds[(i,j)])
                    else:
                        string+="-"
                if string!="-": img = font2.render(string, True, "green")
                else: img = font.render(string, True, "green")
            else:
                string+="#"
                img = font2.render(string, True, "green")
        
            screen.blit(img, (ii, jj))
            string=""
            ii+=14
        jj+=20
        ii=10
    img = font3.render(f"Time: {time}", True, "white")
    img2 = font3.render(f"Turn on wind directions: ", True, "white")
    screen.blit(img, (50, 600))
    screen.blit(img2, (50, 650))


def wait(x):
    for i in range(x):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                exit()
        switch1.draw()  
        pygame.display.update()
        clock.tick(60)
        
while not done and time<1000:
    time+=1
    movewind()
    survivingelves2={}
    for i in survivingelves:
        location=i
        history=survivingelves[i]
        locations=((location[0]+1,location[1]),(location[0]-1,location[1]),(location[0],location[1]+1),(location[0],location[1]-1),location)
        for j in locations:
            if j not in setofwinds and j in permissiblelocations:
                elf=(j,history)
                if j==(len(data[-1])-2,len(data)-1) and history==0:
                    elf=(j,1)
                elif j==(1,0) and history==1:
                    elf=(j,2)
                elif j==(len(data[-1])-2,len(data)-1) and history==2:
                    elf=(j,3)
                    done=True
                if elf[0] in survivingelves2:
                    if elf[1]<survivingelves2[elf[0]]: continue
                survivingelves2[elf[0]]=elf[1]
            
    survivingelves=survivingelves2
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            exit()
    draw()
    switch1.draw()   
    pygame.display.update()
    wait(1)
    
print(done)     
print(time)   

while True:
    wait(10000)
    


(320, 660)
(360, 660)
True
877


SystemExit: 